In [15]:
# Install necessary libraries
!pip install transformers torchaudio

In [14]:

# Import required modules
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torchaudio
from google.colab import files


In [16]:


# Load pre-trained model and processor
model_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:

def predict(audio_path):
    # Load the audio file
    speech_array, sampling_rate = torchaudio.load(audio_path)

    # Resample if necessary
    if sampling_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
        speech_array = resampler(speech_array)

    # Prepare the input features for the model
    inputs = processor(speech_array[0], sampling_rate=16000, return_tensors="pt", padding=True, return_attention_mask=True)

    # Prepare arguments for the model
    input_values = inputs.input_values
    attention_mask = inputs.get('attention_mask', None)

    # Use the model to predict the transcript
    with torch.no_grad():
        model_args = {'input_values': input_values}
        if attention_mask is not None:
            model_args['attention_mask'] = attention_mask
        logits = model(**model_args).logits

    # Decode the model output
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])
    return transcription


In [18]:


# Upload file
uploaded = files.upload()

# Assuming the uploaded file is named correctly after checking
if uploaded:
    file_name = list(uploaded.keys())[0]  # This will use the actual name of the uploaded file
    print("Uploaded file name:", file_name)
    # Use the predict function
    transcription = predict(file_name)
    print("Transcription:", transcription)
else:
    print("No file uploaded.")


Saving harvard.wav to harvard (11).wav
Uploaded file name: harvard (11).wav
Transcription: THE STALE SMELL OF OLD BEER LINGERS IT TAKES HEAT TO BRING OUT THE ODOR A COLD DIP RESTORES HEALTH AND ZEST A SALT PICKLE TASTES FINE WITH HAM TAKOZ AL PASTORE ARE MY FAVORITE A ZESTFUL FOOD IS THE HOT CROSS BUN
